In [ ]:
import cv2
import serial
import time
import numpy as np
from datetime import datetime

In [2]:
from twilio.rest import Client

def SMSmessage():
    # Twilio credentials (update with your actual credentials)
    account_sid = 'AC2a35744fbf116026cdfd82c9a48860c8'
    auth_token = '62b4272c7fc0ffd0ec3db4df0864d98d'

    try:
        client = Client(account_sid, auth_token)

        message = client.messages.create(
            from_='+13614703593',
            body='This is the message content to be sent via SMS.',
            to='+917020821353'  # Replace with the recipient's phone number
        )

        print(f"Message sent successfully. SID: {message.sid}")

    except Exception as e:
        print(f"Error sending SMS: {e}")

/Users/prasadlokhande/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
def send_command(command):
    """
    Send a command to the Arduino, ensuring no redundant commands are sent.
    """
    try:
        ard.write(command.encode())
    except Exception as e:
        print(f"Error sending command to Arduino: {e}")


In [4]:
import cv2
import serial
import time
import sys

# Initialize serial communication with Arduino (update port as needed)
try:
    ard = serial.Serial("/dev/cu.usbserial-1120", 9600, timeout=1)  # Update to your port
    time.sleep(2)  # Allow time for Arduino to initialize
    print("Serial communication initialized.")
except Exception as e:
    print(f"Error initializing serial communication: {e}")
    sys.exit(1)

# Load Haar cascades for face detection (frontal and profile)
face_cascade_frontal = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml')
face_cascade_profile = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_profileface.xml')

if face_cascade_frontal.empty() or face_cascade_profile.empty():
    print("Error loading Haar cascades. Check the file paths.")
    sys.exit(1)

# Start video capture
vid = cv2.VideoCapture(0)
if not vid.isOpened():
    print("Error: Unable to access the camera.")
    sys.exit(1)

# Define frame center thresholds
FRAME_WIDTH = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
FRAME_HEIGHT = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
X_CENTER = FRAME_WIDTH // 2
Y_CENTER = FRAME_HEIGHT // 2
CENTER_THRESHOLD = 50

Serial communication initialized.


In [5]:
def preprocess_frame(frame):
    """
    Preprocess the frame for noise reduction and improved face detection.
    """
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    # Apply bilateral filter to preserve edges while reducing noise
    filtered = cv2.bilateralFilter(blurred, 9, 75, 75)
    return filtered

In [6]:


def apply_thresholding(frame, step=100):
    """
    Apply thresholding to the frame if it is too dark to detect faces.
    Incrementally adjust the threshold from 0 to 255 in steps.
    """
    for threshold in range(0, 256, step):
        _, thresh_frame = cv2.threshold(frame, threshold, 255, cv2.THRESH_BINARY)
        faces_frontal = face_cascade_frontal.detectMultiScale(thresh_frame, scaleFactor=1.1, minNeighbors=5, minSize=(80, 80))
        faces_profile = face_cascade_profile.detectMultiScale(thresh_frame, scaleFactor=1.1, minNeighbors=5, minSize=(80, 80))

        if len(faces_frontal) > 0 or len(faces_profile) > 0:
            return thresh_frame, faces_frontal if len(faces_frontal) > 0 else faces_profile

    return None, []

In [ ]:
def detect_body_like_shapes(frame):
    """
    Detect human-like body shapes based on contours and area filtering.
    """
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Threshold the image
    _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        # Calculate area and filter based on a reasonable size range
        area = cv2.contourArea(contour)
        if 5000 < area < 50000:  # Adjust these thresholds as needed
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = w / float(h)

            # Check for human-like aspect ratio
            if 0.2 < aspect_ratio < 0.7:  # Example: typical torso ratio
                return x, y, w, h

    return None


try:
    while True:
        # Read the current frame
        ret, frame = vid.read()
        if not ret:
            print("Error: Unable to read from the camera.")
            break

        # Preprocess the frame
        processed_frame = preprocess_frame(frame)

        # Detect faces using both frontal and profile cascades
        faces_frontal = face_cascade_frontal.detectMultiScale(processed_frame, scaleFactor=1.1, minNeighbors=5, minSize=(80, 80))
        faces_profile = face_cascade_profile.detectMultiScale(processed_frame, scaleFactor=1.1, minNeighbors=5, minSize=(80, 80))

        # Combine detected faces
        faces = faces_frontal if len(faces_frontal) > 0 else faces_profile

        # If no faces detected, apply thresholding and detect body-like shapes
        if len(faces) == 0:
            processed_frame, faces = apply_thresholding(processed_frame)

            if len(faces) == 0:  # Still no faces detected
                body_coords = detect_body_like_shapes(frame)
                if body_coords:
                    x, y, w, h = body_coords
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                    filename = f"./img/body_{timestamp}.jpg"
                    cv2.imwrite(filename, frame)
                    print(f"Body image saved as {filename}")

        # Process only the first detected face
        if len(faces) > 0:
            # Notify Arduino that a face is detected
            send_command('F')  # 'F' command indicates face detected
            # SMSmessage();

            x, y, w, h = faces[0]  # Select the first face
            # Draw a rectangle around the detected face
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # Calculate the center of the face
            Xpos = x + w // 2
            Ypos = y + h // 2

            # Check the position of the face relative to the frame center
            if Xpos > X_CENTER + CENTER_THRESHOLD:
                send_command('L')  # Face is to the right of center
            elif Xpos < X_CENTER - CENTER_THRESHOLD:
                send_command('R')  # Face is to the left of center
            else:
                send_command('S')  # Face is horizontally centered

            if Ypos > Y_CENTER + CENTER_THRESHOLD:
                send_command('D')  # Face is below the center
            elif Ypos < Y_CENTER - CENTER_THRESHOLD:
                send_command('U')  # Face is above the center
            else:
                send_command('S')  # Face is vertically centered

            # Save the image with the detected face
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"./thief_{timestamp}.jpg"
            cv2.imwrite(filename, frame)
            print(f"Image saved as {filename}")

            time.sleep(0.01)  # Short delay to allow Arduino to process commands

        # Display the frame with the rectangle around the detected face
        cv2.imshow('Face Tracking', frame)

        # Exit the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except KeyboardInterrupt:
    print("Exiting due to keyboard interrupt.")
finally:
    # Release resources
    vid.release()  # Stop video capture
    cv2.destroyAllWindows()  # Close all OpenCV windows
    if 'ard' in locals() and ard.is_open:
        ard.close()  # Close the serial communication

Body image saved as ./img/body_20241215_010123.jpg


2024-12-15 01:01:23.475 Python[90389:2852173] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.


Body image saved as ./img/body_20241215_010124.jpg
Body image saved as ./img/body_20241215_010124.jpg
Body image saved as ./img/body_20241215_010124.jpg
Body image saved as ./img/body_20241215_010125.jpg
Body image saved as ./img/body_20241215_010125.jpg
Body image saved as ./img/body_20241215_010126.jpg
Body image saved as ./img/body_20241215_010126.jpg
Body image saved as ./img/body_20241215_010126.jpg
Body image saved as ./img/body_20241215_010127.jpg
Body image saved as ./img/body_20241215_010127.jpg
Body image saved as ./img/body_20241215_010128.jpg
Body image saved as ./img/body_20241215_010128.jpg
Body image saved as ./img/body_20241215_010128.jpg
Body image saved as ./img/body_20241215_010129.jpg
Body image saved as ./img/body_20241215_010129.jpg
Body image saved as ./img/body_20241215_010130.jpg
Body image saved as ./img/body_20241215_010130.jpg
Body image saved as ./img/body_20241215_010130.jpg
Body image saved as ./img/body_20241215_010131.jpg
Body image saved as ./img/body_

: 

In [ ]:
send_command('U')
send_command('U')
send_command('U')

send_command('U')

send_command('U')
send_command('U')
send_command('U')
send_command('U')

send_command('U')
send_command('U')
send_command('U')
send_command('U')
send_command('U')
